In [2]:
import pandas as pd
import json
import random
import csv
from collections import defaultdict
import pickle
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import random as rn
rn.seed(13)

In [3]:
data_dir = '/home/ckm/visualqatickets/superuser/Data'

In [4]:
def string_sentence(i,qr):
    return qr.loc[i,'Title'] + ' '  + ' '.join(qr.loc[i,'Tags']) + ' '  + qr.loc[i,'Text']

In [5]:
def both_related(qr):
    for i in qr.dropna(subset=['Linked']).index.values:
        if(len(qr.loc[i,'Linked'])!=0):
            for j in qr.loc[i,'Linked']:
                if(j in qr.index.values):
                    if((i in qr.loc[j,'Linked'])==0):
                        qr.loc[j,'Linked'].append(i)
    return qr

In [6]:
with open(data_dir+'/nosw/qr.json','r') as a:
    qr = pd.read_json(a,orient='table')

In [7]:
with open(data_dir+'/pandas/split.txt','rb') as a:
    train_qr = pickle.load(a)
    dev_qr = pickle.load(a)
    test_qr = pickle.load(a)

In [65]:
train_text = train_qr['Text'].tolist()
train_title = train_qr['Title'].tolist()
train_tag = [' '.join(j) for j in train_qr['Tags'].tolist()]

In [70]:
# # find max length of input text/image top

# # max_length of image_top = 12
lengths = []
mx = 0
cnt = 0
# tokenised = []
# avg = 0
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

for idx in range(len(train_text)):
#     text = re.sub(r'(\s)emoji\w+', '', id)
    text = '[CLS] ' + train_title[idx] + train_tag[idx] + train_text[idx] + ' [SEP]'
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#     # while len(indexed_tokens) < 360:
#             # indexed_tokens.append(0)
    cnt += 1
#     # tokenised.append(indexed_tokens)
#     # avg += len(indexed_tokens)
    # if (cnt%1000 == 0):
#         # save_file('./test_text_tokenised', tokenised)
    if (cnt%1000 == 0):
    	print(mx, cnt)
    mx = max(mx, len(indexed_tokens))
    lengths.append(len(indexed_tokens))
# print(len(train_text))

# # print(avg/len(image_top))


5894 1000
7994 2000
7994 3000
7994 4000
7994 5000
7994 6000
11140 7000
11140 8000
11140 9000
11140 10000
11140 11000
11140 12000
11140 13000
11140 14000
11140 15000
11140 16000
11140 17000
11140 18000
11140 19000
11140 20000
11140 21000
11140 22000
11140 23000
11140 24000
11140 25000
11140 26000
11140 27000
11140 28000
11140 29000
11140 30000
11140 31000
11140 32000
11140 33000
11140 34000
11140 35000
11140 36000
11140 37000
11140 38000
11140 39000
11140 40000
11140 41000
11140 42000
11140 43000
11140 44000
11140 45000
11140 46000
11140 47000
11140 48000
11140 49000
11140 50000
11140 51000
11140 52000
11140 53000
11140 54000
11140 55000
11140 56000
11140 57000
11140 58000
11140 59000
11140 60000
11140 61000


In [77]:
import matplotlib.pyplot as plt

x = np.array(lengths)
print(np.sum(x<250))
# plt.hist(lengths,[0,20,40,60,80,100,120,140,160,180,200],range = [0,200])
# plt.show()

52225


In [7]:
def check_str(a):
    if(type(a) is str):
        a = [a]
    return a

In [8]:
def ck_same_tags(i,j):
    tg1 = check_str(qr.loc[i,'Tags'])
    tg2 = check_str(qr.loc[j,'Tags'])
    for j in tg1:
        if j in tg2:
            return True
    return False

In [9]:
def create_pairs(qr,indices,indices_rel,all_pairs = list()):
    pairs = list()
    pos = list()
    neg = list()
    score = list()
    for i in indices_rel:
        for j in qr.loc[i,'Related']:
            if(not([i,j] in pos) and j in qr.index.values):
                pos.append([i,j])
                pos.append([j,i])
                pairs.append([i,j])
                pairs.append([j,i])
                score.append(1)
                score.append(1)
                id2 = rn.choice(qr.index.values)
                while(ck_same_tags(i,id2) or id2 in qr.loc[i,'Related'] or [i,id2] in neg or [i,id2] in all_pairs):
                    id2 = rn.choice(qr.index.values)
                neg.append([i,id2])
                neg.append([id2,i])
                pairs.append([i,id2])
                pairs.append([id2,i])
                score.append(0)
                score.append(0)
                if(len(pos)%1000==0):
                    print(f'{len(pos)} pairs have been created')
    print(f'Overall {len(pos)} have been created')
    return pos,neg,pairs,score

In [10]:
train_pos,train_neg,train_all,train_all_score = create_pairs(train_qr,qr.index.values,(train_qr.dropna(subset=['Related']).index.values))

1000 pairs have been created
2000 pairs have been created
3000 pairs have been created
4000 pairs have been created
5000 pairs have been created
6000 pairs have been created
7000 pairs have been created
8000 pairs have been created
9000 pairs have been created
10000 pairs have been created
11000 pairs have been created
12000 pairs have been created
13000 pairs have been created
14000 pairs have been created
15000 pairs have been created
16000 pairs have been created
17000 pairs have been created
18000 pairs have been created
19000 pairs have been created
20000 pairs have been created
21000 pairs have been created
22000 pairs have been created
23000 pairs have been created
24000 pairs have been created
25000 pairs have been created
26000 pairs have been created
27000 pairs have been created
28000 pairs have been created
29000 pairs have been created
30000 pairs have been created
31000 pairs have been created
32000 pairs have been created
33000 pairs have been created
34000 pairs have be

In [23]:
folder_quora = '/home/ckm/visualqatickets/superuser/Data'
with open(folder_quora+'/pairs_related/data_pos_neg.txt','rb') as a:
    train_data_pos = pickle.load(a)
    train_data_neg = pickle.load(a)
    dev_data_pos = pickle.load(a)
    dev_data_neg = pickle.load(a)
    test_data_pos = pickle.load(a)
    test_data_neg = pickle.load(a)

In [11]:
dev_pos,dev_neg,dev_all,dev_all_score = create_pairs(pd.concat([train_qr,dev_qr]),qr.index.values,(dev_qr.dropna(subset = ['Related'])).index.values,train_pos+train_neg)

1000 pairs have been created
2000 pairs have been created
3000 pairs have been created
4000 pairs have been created
5000 pairs have been created
6000 pairs have been created
7000 pairs have been created
8000 pairs have been created
9000 pairs have been created
10000 pairs have been created
11000 pairs have been created
Overall 11132 have been created


In [12]:
test_pos,test_neg,test_all,test_all_score = create_pairs(pd.concat([train_qr,test_qr]),qr.index.values,(test_qr.dropna(subset = ['Related'])).index.values,train_pos+train_neg+dev_pos+dev_neg)

1000 pairs have been created
2000 pairs have been created
3000 pairs have been created
4000 pairs have been created
5000 pairs have been created
6000 pairs have been created
7000 pairs have been created
8000 pairs have been created
9000 pairs have been created
10000 pairs have been created
Overall 10808 have been created


In [13]:
os.makedirs(data_dir+'/pairs_related',exist_ok=True)

In [14]:
with open(data_dir+'/pairs_related/data_pos_neg.txt','wb') as a:
    pickle.dump(train_pos,a)
    pickle.dump(train_neg,a)
    pickle.dump(dev_pos,a)
    pickle.dump(dev_neg,a)
    pickle.dump(test_pos,a)
    pickle.dump(test_neg,a)

In [15]:
train_data = train_pos+train_neg
train_score = [1]*len(train_pos) + [0]*len(train_neg)
train_data,train_score = shuffle(train_data,train_score)
dev_data = dev_pos+dev_neg
dev_score = [1]*len(dev_pos) + [0]*len(dev_pos)
dev_data,dev_score = shuffle(dev_data,dev_score)
test_data = test_pos+test_neg
test_score = [1]*len(test_pos)+[0]*len(test_pos)
test_data,test_score = shuffle(test_data,test_score)

In [16]:
with open(data_dir+'/pairs_related/train_dev_test.txt','wb') as a:
    pickle.dump(train_data,a)
    pickle.dump(train_score,a)
    pickle.dump(dev_data,a)
    pickle.dump(dev_score,a)
    pickle.dump(test_data,a)
    pickle.dump(test_score,a)

In [17]:
os.makedirs(data_dir+'/pairs_related/siamese',exist_ok=True)

In [18]:
with open(data_dir+'/pairs_related/siamese/train_dev_test.txt','wb') as a:
    pickle.dump(train_all,a)
    pickle.dump(train_all_score,a)
    pickle.dump(dev_all,a)
    pickle.dump(dev_all_score,a)
    pickle.dump(test_all,a)
    pickle.dump(test_all_score,a)